In [94]:
#This is just to increase my Jupyter Notebook size to fill my screen.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [185]:
#Importing Necessary Pre-requisites
import sys
import os
import re
import glob
import codecs
import numpy as np
import pandas as pd
from collections import defaultdict
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stopList = stopwords.words('english')
from collections import Counter

In [96]:
# Creating a method to open the file such that it can be read by Python
def readFile(filename):
    with open(filename, "r", encoding='mac_roman') as f:
        content = f.read()
    f.close()

    return content

In [136]:
# Defining our directory and opening the corpus

print('Please input filepath of cran corpus as located on YOUR LOCAL MACHINE')
#cranCorpus="C:\\Users\\J\\Desktop\\Data Science\\Spring 2022\\DASC 6030\\cranfield-corpus\\cran.all.1400"
cranCorpus=input()
#"C:\\Users\\J\\Desktop\\Data Science\\Spring 2022\\DASC 6030\\cranfield-corpus\\cran.all.1400"
print('Please input filepath of cran qry as located on YOUR LOCAL MACHINE')
#cranQRY="C:\\Users\\J\\Desktop\\Data Science\\Spring 2022\\DASC 6030\\cranfield-corpus\\cran.qry"
cranQRY=input()
#"C:\\Users\\J\\Desktop\\Data Science\\Spring 2022\\DASC 6030\\cranfield-corpus\\cran.qry"
print('Please input filepath of cran qrel as located on YOUR LOCAL MACHINE')
#cranRel="C:\\Users\\J\\Desktop\\Data Science\\Spring 2022\\DASC 6030\\cranfield-corpus\\cranqrel"
cranRel=input()
#"C:\\Users\\J\\Desktop\\Data Science\\Spring 2022\\DASC 6030\\cranfield-corpus\\cranqrel"


# Defining Porter Stemmer

ps = PorterStemmer()

Please input filepath of cran corpus as located on YOUR LOCAL MACHINE
Please input filepath of cran qry as located on YOUR LOCAL MACHINE
Please input filepath of cran qrel as located on YOUR LOCAL MACHINE


In [143]:
# Defining a variable to identify the index delimiter '.I'
docID = re.compile('.\.I.')

# Method to open the file and retrieve each of the document indexes in the Cran Corpus
def getData(filePath, iD):
    with open (filePath, 'r') as f:
        text = f.read().replace('\n'," ")
        lines = re.split(iD, text)
        lines.pop(0)
    return lines

# Retreiving document IDs from the Cran Corpus and Cran Query and storing them each in a variable. 
txtList = getData(cranCorpus, docID)
qryList = getData(cranQRY, docID)

#print(len(txtList))
#should return 1400
#print(len(qryList))
#should return 225

numDoc = (len(txtList))

In [144]:
# Method for normalization and stemming of Corpus
def normalizeTxt(dirtyTxt):
    # Converting text to lower case
        lines = dirtyTxt.lower()
    
    # Removing punctuations
        lines = re.sub('[^A-Za-z]+', ' ', lines)
    
    # Removing whitespaces to generate tokens
        tokens = line.split()
        
    # removing stop words from the tokens
        cleanTokens = [word for word in tokens if word not in stopList]
    
    # Tokenization
        tokenList = word_tokenize(dirtyTxt)
    
    # Checking to verify we only get alphabetic characters
        tokenList = [word for word in tokenList if word.isalpha()]
    
    # Stemming
        stemedList = [ps.stem(word) for word in tokenList]
        
    # checking for stopwords again
        twiceCleanedTokens = [word for word in stemedList if word not in stopList]
        
        return twiceCleanedTokens

In [145]:
# Creating the parser and dictionary based on the corpus delimiters (excluding Index)
textPars = re.compile('\.[A,B,T,W]')

# Creating the dictionary to store the corpus information based
txtData = defaultdict(dict)


# Method to parse the text and store the ID, Title, Author, Publication Information, and Text Body based on the delimiters ".I, .T, .A, .B, .W"
# Performs normalization on Title and Text Fields

for line in txtList:
    entries = re.split(textPars,line)
    id = entries[0].strip()
    title = entries[1]
    author = entries[2]
    publicationInfo = entries[3]
    text = entries[4]
    txtData[id]['title'] = normalizeTxt(title)
    txtData[id]['author'] = author
    txtData[id]['publicationInfo'] = publicationInfo
    txtData[id]['text'] = normalizeTxt(text)
    
    # Uncomment to test integerity of normalize method on either text or title.
    
    #print(id, txtData[id]['text'])
    #print(txtData[id]['title'])

In [183]:
# Creating the parser and dictionary based on the query delimiters (excluding Index)
qryPars = re.compile('\.[I,W]')

# Creating the dictionary to store the corpus information based
qryData = defaultdict(dict)


# Method to parse the text and store the ID and query based on the delimiters ".I, .W"
# Performs normalization on query Field

for line in qryList:
    entries = re.split(qryPars,line)
    id = entries[0].strip()
    qryText = entries[1]
    qryData[id]['qryText'] = normalizeTxt(qryText)
    
    # Uncomment to test integerity of normalize method on query
    
    #print(id, qryData[id]['qryText'])
    #print(qryData[id]['qryText'])

In [101]:
# Parsing the queries posted on the corpus
qryPars = re.compile('\.[W]')
qryData = defaultdict(dict)
 
for n in range(0,len(qryList)-1):
  line = qryList[n+1]
  _ , question = re.split(qryPars,line)
  qryData[n+1]['question'] = question

#To print all queries
  # print(qryData) 

In [207]:
# Creation of dictionary to store relevence assessment
cranRelTxt = open(cranRel)
cranNumPy = np.loadtxt(cranRelTxt, dtype = int)
cranRelDic = defaultdict(list)
for row in cranNumPy:
    cranRelDic[row[0]].append(tuple(row[1:]))
    #print(cranRelDic)

In [103]:
cran_rel_data = open(cranRel)
cran_np = np.loadtxt(cran_rel_data, dtype=int)
 
cran_rel_rat = defaultdict(list)
for row in cran_np:
    cran_rel_rat[row[0]].append(tuple(row[1:]))

In [197]:
#successfully creates list of text tokens
dictTokens = []
for id in txtData:
    dictTokens.append(txtData[id]['text'])
#print(dictTokens)

#successfully creates list of title tokens
titleTokens = []
for id in txtData:
    titleTokens.append(txtData[id]['title'])
#print(titleTokens)

#successfully creates list of query tokens
qryTokens = []
for id in qryData:
    qryTokens.append(qryData[id]['qryText'])
#print(qryTokens)

In [198]:
# create a dictionary of key-value pairs where tokens are keys and their occurrence in the corpus the value for text
textDF = {}
for id in txtData:
    tokens = txtData[id]['text']
    for w in tokens:
        try:
            # add token as key and doc number as value is chained
            textDF[w].add(id)
        except:
            # to handle when a new token is encountered
            textDF[w] = {id}

# convert to number of occurrences of the token from list of documents where token occurs for text
for id in textDF:
    textDF[id] = len(textDF[id])

#print(textDF)

# create a dictionary of key-value pairs where tokens are keys and their occurrence in the corpus the value for title
titleDF = {}
for id in txtData:
    tokens = txtData[id]['title']
    for w in tokens:
        try:
            # add token as key and doc number as value is chained
            titleDF[w].add(id)
        except:
            # to handle when a new token is encountered
            titleDF[w] = {id}

# convert to number of occurrences of the token from list of documents where token occurs for text
for id in titleDF:
    titleDF[id] = len(titleDF[id])

#print(titleDF)

In [160]:
# count number of unique words in the corpus text
textVocabSize = len(textDF)
#print(textVocabSize)

# create vocabulary list of all unique words in text
textVocab = [term for term in textDF]
#print(textVocab)

# count number of unique words in the corpus title
titleVocabSize = len(titleDF)
#print(titleVocabSize)

# create vocabulary list of all unique words in title
titleVocab = [term for term in titleDF]
#print(titleVocab)

In [200]:
# creating dictionary to store tf-idf values for each term in the vocabulary for text
doc = 0
textTfIdf = {}

for id in txtData:
    tokens = txtData[id]['text']

    # counter object to efficiently count number of occurrence of a term in a particular document for text
    counter = Counter(tokens)
    token_count = len(tokens)

    for token in np.unique(tokens):
        # counting occurence of term in object using counter object
        tf = counter[token] / token_count
        # retrieving df values from DF dictionary
        df = textDF[token] if token in textVocab else 0

        # adding 1 to numerator & denominator to avoid divide by 0 error
        idf = np.log((numDoc + 1) / (df + 1))

        textTfIdf[doc, token] = tf * idf

    doc += 1

#print(textTfIdf)

In [169]:
# creating dictionary to store tf-idf values for each term in the vocabulary for title
doc = 0
titleTfIdf = {}

for id in txtData:
    tokens = txtData[id]['title']

    # counter object to efficiently count number of occurrence of a term in a particular document for title
    counter = Counter(tokens)
    token_count = len(tokens)

    for token in np.unique(tokens):
        # counting occurence of term in object using counter object
        tf = counter[token] / token_count
        # retrieving df values from DF dictionary
        df = titleDF[token] if token in titleVocab else 0

        # adding 1 to numerator & denominator to avoid divide by 0 error
        idf = np.log((numDoc + 1) / (df + 1))

        titleTfIdf[doc, token] = tf * idf

    doc += 1

#print(titleTfIdf)

In [203]:
#initializing empty vector of vocabulary size
D = np.zeros((numDoc, textVocabSize))

#creating vector of tf-idf values
for i in textTfIdf:
    ind = textVocab.index(i[1])
    D[i[0]][ind] = textTfIdf[i]

#confirm number of vectors and vector preview
#print(len(D))
#print(D)

In [178]:
def gen_vector(tokens):
    """To create a vector (with repsect to the vocabulary) of the tokens passed as input
    
    Arguments:
        tokens {list} -- list of tokens to be converted
    
    Returns:
        numpy.ndarray -- vector of tokens
    """
    Q = np.zeros((len(textVocab)))
    
    counter = Counter(tokens)
    tokenCount = len(tokens)

    qryWeights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/tokenCount
        df = textDF[token] if token in textVocab else 0
        idf = np.log((numDoc+1)/(df+1))

        try:
            ind = textVocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [179]:
def cosine_sim(x, y):
    """To calculate cosine similarity between 2 vectors.
    
    Arguments:
        x {numpy.ndarray} -- vector 1
        y {numpy.ndarray} -- vector 2
    
    Returns:
        numpy.float64 -- cosine similarity between vector 1 & vector 2
    """
    cos_sim = np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y))
    
    return cos_sim

In [180]:
def cosine_similarity(k, query):
    """To determine a ranked list of top k documents in descending order of their
    cosine similarity with the query
    
    Arguments:
        k {integer} -- top k documents to retrieve from 
        query {string} -- query whose cosine similarity is to be computed with the corpus
    
    Returns:
        numpy.ndarray -- list of top k cosine similarities between query and corpus of documents
    """

    tokens = query.split()
      
    d_cosines = []
    
    # vectorize the input query tokens
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    if k == 0:
        # k=0 to retrieve all documents in descending order
        out = np.array(d_cosines).argsort()[::-1]
        
    else:
        # to retrieve the top k documents in descending order    
        out = np.array(d_cosines).argsort()[-k:][::-1]
    
    return out

In [181]:
def list_of_docs(k):
    """To generate a ranked list of k top documents in descending order of their cosine similarity 
    calculated against the queries. Output is a list of (query id, document id) pairs.
    
    If k=0 is given as input then list of all documnets in descending order is returned.
    
    Arguments:
        k {integer} -- number of top documents to be retrieved
    
    Returns:
        list -- list of documents in descending order of their cosine similarity
    """
    cos_sims = []
    for i in range(len(qryList)):
        cs = [i, cosine_similarity(k, qryList[i])]
        cos_sims.append(cs)
        
    return cos_sims

In [205]:
#to get list of all documents
#May result in runtime error on local machine!
#May resolve with from scipy.special import logsumexp ...?
#cos_sim = np.dot(x, y) / (np.linalg.norm.logsumexp(x)*np.linalg.norm.logsumexp(y)) ...?

#no_of_top=0
#list_of_docs(no_of_top)

In [190]:
# retrieving relevance values from cranqrel
# Creating the parser and dictionary based on the cranqrel delimiters
textPars = re.compile('\.[A,B,T,W]')

In [ ]:
# Work completed so far
# 1) Created a normalize method
# 2) Created Dictionary of doc ID, non-normalized author and publication Info, and normalized title and text
# 3) Created Dictionary for each query in the provided query doc.
# 4) Created Dictionary to store relevence assessments from provided relevence document
# 5) Create a dicitionary of all tokenized words and their frequency
#    a)  Need dictionary for text
#    b)  Need diciontary for title
# 6) Calculate DF values for each term in the vocabulary
# 7) Calcualte tf-idf vlaues for each term in the vocabulary
# 8) Form document vectors using tf-idf values

In [18]:
# Remaining work to be completed

# 5) Develop user interface which allows user to boost vlaues
# 6) Compute Compute the similarity of the query with the corpus documents to produce top k results
# 7) Show titles of ranked documetns that are relvent to query. (May also use abstract for this)